# Modifying Existing .lfcalc Files

This notebook demonstrates how to read, query, modify, and save existing Leapfrog calculation sets. You'll learn:
- Reading .lfcalc files into Python
- Querying and filtering calculations
- Modifying existing calculations
- Adding new calculations
- Removing calculations
- Batch modifications
- Version control workflows

In [ ]:
import pollywog as pw
from pollywog.core import CalcSet, Number, Category, Variable, Filter, If
from pollywog.helpers import Scale, WeightedAverage
from pollywog.display import display_calcset, set_theme

set_theme("light")

## 1. Reading .lfcalc Files

Load existing calculation sets from Leapfrog.

In [ ]:
# First, let's create an example .lfcalc file to work with
original = CalcSet([
    Number("Au_est_v1", "[Au_kriged]", comment_equation="Original estimate"),
    Number("Au_diluted_v1", "[Au_est_v1] * 0.90", comment_equation="10% dilution"),
    Number("Cu_est_v1", "[Cu_kriged]", comment_equation="Original estimate"),
    Number("Cu_diluted_v1", "[Cu_est_v1] * 0.92", comment_equation="8% dilution"),
    Number("tonnage", "[volume] * [density]", comment_equation="Block tonnage"),
])

original.to_lfcalc("original_estimates.lfcalc")
print("✅ Created original_estimates.lfcalc\n")

# Now read it back
loaded = CalcSet.read_lfcalc("original_estimates.lfcalc")

print(f"Loaded {len(loaded.items)} calculations:")
for item in loaded.items:
    print(f"  - {item.name}: {item.expression}")

display_calcset(loaded)

## 2. Querying Calculations

Use the `query()` method to filter calculations based on properties.

In [ ]:
# Query by name pattern
au_calcs = loaded.query('name.startswith("Au")')
print(f"Au calculations ({len(au_calcs.items)}):")
for item in au_calcs.items:
    print(f"  - {item.name}")

# Query dilution calculations
diluted = loaded.query('"diluted" in name')
print(f"\nDilution calculations ({len(diluted.items)}):")
for item in diluted.items:
    print(f"  - {item.name}")

# Query by comment
estimates = loaded.query('"estimate" in comment_equation.lower()')
print(f"\nEstimates ({len(estimates.items)}):")
for item in estimates.items:
    print(f"  - {item.name}: {item.comment_equation}")

## 3. Modifying Calculations In-Place

Change properties of existing calculations.

In [ ]:
# Load the file
modified = CalcSet.read_lfcalc("original_estimates.lfcalc")

# Modify dilution factors (change 0.90 to 0.95 for Au)
for item in modified.items:
    if item.name == "Au_diluted_v1":
        # Change expression
        item.expression = ["[Au_est_v1] * 0.95"]
        # Update comment
        item.comment_equation = "5% dilution (updated)"
        print(f"✅ Modified {item.name}")
        print(f"   New expression: {item.expression[0]}")
        print(f"   New comment: {item.comment_equation}")

# Save modified version
modified.to_lfcalc("modified_estimates.lfcalc")
print("\n✅ Saved modified_estimates.lfcalc")

## 4. Adding New Calculations

Add calculations to an existing CalcSet.

In [ ]:
# Load existing file
enhanced = CalcSet.read_lfcalc("original_estimates.lfcalc")

print(f"Original: {len(enhanced.items)} calculations")

# Add recovery calculations
enhanced.items.append(
    Number("Au_recovered", "[Au_diluted_v1] * [recovery_Au]", comment_equation="Apply recovery")
)
enhanced.items.append(
    Number("Cu_recovered", "[Cu_diluted_v1] * [recovery_Cu]", comment_equation="Apply recovery")
)

# Add metal content calculations
enhanced.items.append(
    Number("Au_metal", "[tonnage] * [Au_recovered]", comment_equation="Contained Au (grams)")
)
enhanced.items.append(
    Number("Cu_metal", "[tonnage] * [Cu_recovered] / 100", comment_equation="Contained Cu (tonnes)")
)

# Add classification
enhanced.items.append(
    Category(
        "ore_class",
        If(
            [
                ("[Au_recovered] < 0.3 and [Cu_recovered] < 0.1", "'waste'"),
                ("[Au_recovered] >= 1.0 or [Cu_recovered] >= 0.5", "'high_grade'"),
            ],
            "'low_grade'"
        ),
        comment_equation="Au-Cu classification"
    )
)

print(f"Enhanced: {len(enhanced.items)} calculations")
print(f"Added: {len(enhanced.items) - len(original.items)} new calculations")

# Save enhanced version
enhanced.to_lfcalc("enhanced_estimates.lfcalc")
display_calcset(enhanced)

## 5. Removing Calculations

Filter out unwanted calculations.

In [ ]:
# Load file
trimmed = CalcSet.read_lfcalc("enhanced_estimates.lfcalc")

print(f"Before: {len(trimmed.items)} calculations")

# Remove old version calculations (anything with _v1)
trimmed.items = [item for item in trimmed.items if "_v1" not in item.name]

print(f"After: {len(trimmed.items)} calculations")
print(f"Removed: {len(enhanced.items) - len(trimmed.items)} calculations")

print("\nRemaining calculations:")
for item in trimmed.items:
    print(f"  - {item.name}")

# Save cleaned version
trimmed.to_lfcalc("trimmed_estimates.lfcalc")

## 6. Batch Modifications

Apply systematic changes to multiple calculations.

In [ ]:
# Create a calcset to batch modify
batch_test = CalcSet([
    Number("Au_est", "[Au]"),
    Number("Ag_est", "[Ag]"),
    Number("Cu_est", "[Cu]"),
    Number("Pb_est", "[Pb]"),
    Number("Zn_est", "[Zn]"),
])

print("Original calculations:")
for item in batch_test.items:
    print(f"  {item.name}: {item.expression[0]}")

# Batch modification: Apply 5% dilution to all estimates
for item in batch_test.items:
    if "_est" in item.name:
        # Extract metal name
        metal = item.name.replace("_est", "")
        # Update to diluted calculation
        item.name = f"{metal}_diluted"
        item.expression = [f"[{metal}_est] * 0.95"]
        item.comment_equation = "5% dilution applied"

print("\nAfter batch modification:")
for item in batch_test.items:
    print(f"  {item.name}: {item.expression[0]} ({item.comment_equation})")

## 7. Replacing Calculations by Name

Update specific calculations while preserving others.

In [ ]:
# Load existing
replaced = CalcSet.read_lfcalc("original_estimates.lfcalc")

# Define replacements
replacements = {
    "Au_diluted_v1": Number("Au_diluted_v2", "[Au_est_v1] * 0.95", comment_equation="Updated: 5% dilution"),
    "Cu_diluted_v1": Number("Cu_diluted_v2", "[Cu_est_v1] * 0.94", comment_equation="Updated: 6% dilution"),
}

# Apply replacements
new_items = []
for item in replaced.items:
    if item.name in replacements:
        print(f"✅ Replacing {item.name} with {replacements[item.name].name}")
        new_items.append(replacements[item.name])
    else:
        new_items.append(item)

replaced.items = new_items

# Save
replaced.to_lfcalc("replaced_estimates.lfcalc")
display_calcset(replaced)

## 8. Merging Multiple .lfcalc Files

Combine calculations from different files.

In [ ]:
# Create two separate calcsets
preprocessing = CalcSet([
    Variable("Au_clean", "clamp([Au], 0)"),
    Variable("Cu_clean", "clamp([Cu], 0)"),
])
preprocessing.to_lfcalc("preprocessing.lfcalc")

postprocessing = CalcSet([
    Number("Au_final", "[Au_clean] * 0.95"),
    Number("Cu_final", "[Cu_clean] * 0.92"),
])
postprocessing.to_lfcalc("postprocessing.lfcalc")

# Load and merge
pre = CalcSet.read_lfcalc("preprocessing.lfcalc")
post = CalcSet.read_lfcalc("postprocessing.lfcalc")

# Merge using + operator
merged = pre + post

print(f"Preprocessing: {len(pre.items)} calculations")
print(f"Postprocessing: {len(post.items)} calculations")
print(f"Merged: {len(merged.items)} calculations\n")

# Sort by dependencies
merged_sorted = merged.topological_sort()

print("Merged and sorted:")
for item in merged_sorted.items:
    print(f"  - {item.name}")

merged_sorted.to_lfcalc("merged_workflow.lfcalc")
display_calcset(merged_sorted)

## 9. Version Control Workflow

Best practices for managing calculation sets with version control.

In [ ]:
# Example: Systematic versioning

def create_versioned_workflow(version):
    """Create a versioned calculation workflow."""
    
    # Version-specific parameters
    params = {
        "v1": {"dilution": 0.90, "recovery": 0.85},
        "v2": {"dilution": 0.95, "recovery": 0.88},
        "v3": {"dilution": 0.93, "recovery": 0.90},
    }
    
    p = params[version]
    
    workflow = CalcSet([
        Number(f"Au_diluted_{version}", f"[Au_est] * {p['dilution']}",
               comment_equation=f"Dilution: {(1-p['dilution'])*100:.0f}%"),
        Number(f"Au_recovered_{version}", f"[Au_diluted_{version}] * {p['recovery']}",
               comment_equation=f"Recovery: {p['recovery']*100:.0f}%"),
        Number(f"metal_{version}", f"[tonnage] * [Au_recovered_{version}]",
               comment_equation="Contained metal"),
    ])
    
    return workflow

# Create multiple versions
for ver in ["v1", "v2", "v3"]:
    wf = create_versioned_workflow(ver)
    filename = f"workflow_{ver}.lfcalc"
    wf.to_lfcalc(filename)
    print(f"✅ Created {filename}")

# Load and compare
v1 = CalcSet.read_lfcalc("workflow_v1.lfcalc")
v3 = CalcSet.read_lfcalc("workflow_v3.lfcalc")

print("\nVersion comparison:")
for item_v1, item_v3 in zip(v1.items, v3.items):
    print(f"  v1: {item_v1.name} = {item_v1.expression[0]}")
    print(f"  v3: {item_v3.name} = {item_v3.expression[0]}")
    print()

## 10. Practical Example: Updating Production Workflow

Real-world scenario: Update existing calculations with new parameters.

In [ ]:
# Scenario: Metallurgy team updated recovery factors
# Need to update all recovery calculations

# Create "production" workflow
production = CalcSet([
    Number("Au_est", "[Au_kriged]"),
    Number("Ag_est", "[Ag_kriged]"),
    Number("Cu_est", "[Cu_kriged]"),
    
    Number("Au_rec", "[Au_est] * 0.85", comment_equation="Old recovery: 85%"),
    Number("Ag_rec", "[Ag_est] * 0.75", comment_equation="Old recovery: 75%"),
    Number("Cu_rec", "[Cu_est] * 0.88", comment_equation="Old recovery: 88%"),
    
    Number("tonnage", "[volume] * [density]"),
])

production.to_lfcalc("production_old.lfcalc")

# Load for updating
updated = CalcSet.read_lfcalc("production_old.lfcalc")

# New recovery factors from met team
new_recoveries = {
    "Au": 0.88,  # Updated from 0.85
    "Ag": 0.78,  # Updated from 0.75
    "Cu": 0.90,  # Updated from 0.88
}

# Apply updates
for item in updated.items:
    if "_rec" in item.name:
        metal = item.name.replace("_rec", "")
        if metal in new_recoveries:
            old_recovery = float(item.expression[0].split("*")[1].strip())
            new_recovery = new_recoveries[metal]
            
            item.expression = [f"[{metal}_est] * {new_recovery}"]
            item.comment_equation = f"Updated recovery: {new_recovery*100:.0f}% (was {old_recovery*100:.0f}%)"
            
            print(f"✅ Updated {item.name}: {old_recovery:.2f} → {new_recovery:.2f}")

# Save updated version
updated.to_lfcalc("production_updated.lfcalc")

print("\n✅ Production workflow updated with new recovery factors")
print("📁 Saved as: production_updated.lfcalc")

display_calcset(updated)

## Summary

**Reading and writing:**
```python
calcset = CalcSet.read_lfcalc("file.lfcalc")
calcset.to_lfcalc("output.lfcalc")
```

**Querying:**
```python
subset = calcset.query('name.startswith("Au")')
subset = calcset.query('"dilution" in comment_equation')
```

**Modifying:**
```python
# Modify in-place
for item in calcset.items:
    if condition:
        item.expression = new_expression
        item.comment_equation = new_comment
```

**Adding:**
```python
calcset.items.append(new_calc)
# or
calcset = calcset + other_calcset
```

**Removing:**
```python
calcset.items = [item for item in calcset.items if condition]
```

**Best practices:**
1. ✅ Always use descriptive filenames (include version/date)
2. ✅ Keep backups before modifying production files
3. ✅ Use `topological_sort()` after modifications
4. ✅ Test modified calcsets on small data first
5. ✅ Store Python scripts in version control (Git)
6. ✅ Document changes in comments
7. ✅ Use query to verify changes before saving